In [1]:
%load_ext bigdata
%hive_start
%timeout 300

In [2]:
!hdfs dfs -rm -r -f /tmp/output

Deleted /tmp/output


In [3]:
!hdfs dfs -mkdir /tmp/output

In [6]:
!hdfs dfs -copyFromLocal tbl0.csv /tmp/output/
!hdfs dfs -copyFromLocal tbl1.csv /tmp/output/

copyFromLocal: `/tmp/output/tbl0.csv': File exists


In [7]:
%%hive

DROP TABLE IF EXISTS tbl0;
CREATE TABLE tbl0 (
    c1 INT,
    c2 STRING,
    c3 INT,
    c4 DATE,
    c5 ARRAY<CHAR(1)>,
    c6 MAP<STRING, INT>
)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';

LOAD DATA LOCAL INPATH 'tbl0.csv' INTO TABLE tbl0;
--
DROP TABLE IF EXISTS tbl1;
CREATE TABLE tbl1 (
    c1 INT,
    c2 INT,
    c3 STRING,
    c4 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
LOAD DATA LOCAL INPATH 'tbl1.csv' INTO TABLE tbl1;

DROP TABLE IF EXISTS tbl0;
OK
Time taken: 0.101 seconds
CREATE TABLE tbl0 (
    c1 INT,
    c2 STRING,
    c3 INT,
    c4 DATE,
    c5 ARRAY<CHAR(1)>,
    c6 MAP<STRING, INT>
)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
OK
Time taken: 0.061 seconds
LOAD DATA LOCAL INPATH 'tbl0.csv' INTO TABLE tbl0;
Loading data to table default.tbl0
OK
Time taken: 0.726 seconds
--
DROP TABLE IF EXISTS tbl1;
OK
Time taken: 0.026 seconds
CREATE TABLE tbl1 (
    c1 INT,
    c2 INT,
    c3 STRING,
    c4 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
OK
Time taken: 0.057 seconds
LOAD DATA LOCAL INPATH 'tbl1.csv' INTO TABLE tbl1;
Loading data to table default.tbl1
OK
Time taken: 1.09 seconds


In [11]:
%%hive

DROP TABLE IF EXISTS dos_join;

CREATE TABLE dos_join
AS

SELECT t0.c1,
       t0.c2,
       tt1.val

FROM tbl0 t0

JOIN (SELECT c1,
            key,
            val
      FROM  tbl1
      LATERAL VIEW explode(c4) tbl1 AS key, val
     ) tt1

ON (tt1.c1 = t0.c1 AND tt1.key = t0.c2)
;

INSERT OVERWRITE DIRECTORY '/tmp/output'
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
SELECT * FROM dos_join;

DROP TABLE IF EXISTS dos_join;
OK
Time taken: 0.122 seconds
CREATE TABLE dos_join
AS
SELECT t0.c1,
       t0.c2,
       tt1.val
FROM tbl0 t0
JOIN (SELECT c1,
            key,
            val
      FROM  tbl1
      LATERAL VIEW explode(c4) tbl1 AS key, val
     ) tt1
ON (tt1.c1 = t0.c1 AND tt1.key = t0.c2)
;
Query ID = root_20200119004211_aed595db-a47d-49e6-a691-c1e63db01998
Total jobs = 1
SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/local/hive/lib/log4j-slf4j-impl-2.6.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/local/hadoop/share/hadoop/common/lib/slf4j-log4j12-1.7.10.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
2020-01-19 00:42:19	Starting to launch local task to process map join;	maximum memory = 477626368
2020-01-19 00:42:21	Dump the sid

In [12]:
!hdfs dfs -copyToLocal /tmp/output output;

In [15]:
%%hive
DROP TABLE dos_join;

KeyboardInterrupt: 

In [14]:
%hive_quit